In [18]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

## Load Data

In [19]:
pgd = ph.phyloData()

the data of phylo


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [20]:
pgd.search_name("primates")

,name,id
0,Primates,15963


In [21]:
#pgd.fetch_tol_data(15963, limit=25000)

In [22]:
#pgd.add_time()
#pgd.df.head()

### 2) load from raw file

In [23]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [24]:
#pgd.write_prep_data("data/mammal_df.csv", "data/mammal_links.txt")

### 3) load from prepared files

In [25]:
#pgd.load_prep_data("data/Mammalia-15040-df.csv")
pgd.load_prep_data("data/Chordata-2499-df.csv")


### return data for plotting

In [26]:
df = pgd.return_data()
print(df.shape)
df.head()

(10395, 11)


,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End
0,2466,1,0,2499,Chordata,2,0,-0.794184,0.918448,558.0,0.0
1,2499,2,2,14819,Calcichordata,0,0,-0.091174,0.033661,NaN,NaN
2,2499,2,0,14821,Urochordata,8,0,-1.014761,0.366714,555.0,0.0
3,14821,3,0,114489,Octacnemidae,0,0,-0.066112,-0.370996,NaN,NaN
4,14821,3,0,114490,Phlebobranchiata,4,0,-0.344893,-0.201929,NaN,NaN


## Plot Data

In [27]:
pgp = ph.phyloGraph(df)

the plotting of phylo


In [28]:
pgp.search_name('Mamma')

                 name     id
164          Mammalia  15040
1734  Mammalodontidae  16029


In [29]:
pgp.search_name('rrhini')

              name     id
607    Platyrrhini  16292
624     Catarrhini  16293
689  Strepsirrhini  16295


In [30]:
root_node=15040
focus_node = 16293
#focus_node = 'all'

In [31]:
pgp.create_plot_data(root=root_node, focus=focus_node, add_links=True)
#pgp.create_plot_data(root=root_node, focus=focus_node, add_links=True, , Z_dim='depth', Z_dim_mult=-1)

/Users/Seth/Documents/tolGraph/phyloGraph.py:426: RuntimeWarning:

All-NaN slice encountered

/Users/Seth/Documents/tolGraph/phyloGraph.py:492: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting 16293


In [33]:
#pgp.plot_data
#pgp.render_plot()
pgp.render_plot(publish=True, filename="Mammals")

In [13]:
#pgp.open_plot()